Libraries importing

In [15]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from rl.agents.dqn import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

Helper Functions to build the DQN and Feature Engineering (timestamp)

In [16]:
# Helper function to build a Deep Q-Network (DQN) model
def build_dqn_model(input_shape, nb_actions):
    model = Sequential()
    model.add(Dense(64, input_shape=input_shape, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(nb_actions, activation='linear'))  # Output layer for regression-like tasks
    return model

# Function to build and return a DQN agent
def build_dqn_agent(model, nb_actions):
    memory = SequentialMemory(limit=50000, window_length=1)
    policy = BoltzmannQPolicy()
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                   nb_actions=nb_actions, nb_steps_warmup=10, target_model_update=1e-2)
    dqn.compile(Adam(lr=1e-3), metrics=['mae'])
    return dqn

# Feature engineering (include timestamp features)
def feature_engineering(df):
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['hour'] = df['timestamp'].dt.hour
    df['day'] = df['timestamp'].dt.day
    df['month'] = df['timestamp'].dt.month
    df['sin_hour'] = np.sin(2 * np.pi * df['hour'] / 24)
    df['cos_hour'] = np.cos(2 * np.pi * df['hour'] / 24)
    return df


Dataset loading

In [17]:
# Load the dataset and prepare features
df = pd.read_csv('IGI.csv')  # Replace with your actual data file path
df = feature_engineering(df)

# Features and target
X = df[['Glucose_old', 'Glucose_new', 'sin_hour', 'cos_hour']].values
y = df['Insulin'].values

# Split the data into training and testing sets and keep track of indices
X_train, X_test, y_train, y_test, train_idx, test_idx = train_test_split(X, y, df.index, test_size=0.2, random_state=42)


Training and evaluation of the other algorithms

In [18]:
# Initialize result storage
results = {'Algorithm': [], 'RMSE': [], 'MAE': []}

# 1. Linear Regression
lr = LinearRegression()
lr.fit(X_train, y_train)
lr_pred = lr.predict(X_test)
lr_rmse = np.sqrt(mean_squared_error(y_test, lr_pred))
lr_mae = mean_absolute_error(y_test, lr_pred)
results['Algorithm'].append('Linear Regression')
results['RMSE'].append(lr_rmse)
results['MAE'].append(lr_mae)

# 2. Random Forest Regressor
rf = RandomForestRegressor()
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)
rf_rmse = np.sqrt(mean_squared_error(y_test, rf_pred))
rf_mae = mean_absolute_error(y_test, rf_pred)
results['Algorithm'].append('Random Forest')
results['RMSE'].append(rf_rmse)
results['MAE'].append(rf_mae)

# 3. PCA + Linear Regression
pca = PCA(n_components=2)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)
pca_lr = LinearRegression()
pca_lr.fit(X_train_pca, y_train)
pca_lr_pred = pca_lr.predict(X_test_pca)
pca_lr_rmse = np.sqrt(mean_squared_error(y_test, pca_lr_pred))
pca_lr_mae = mean_absolute_error(y_test, pca_lr_pred)
results['Algorithm'].append('PCA + Linear Regression')
results['RMSE'].append(pca_lr_rmse)
results['MAE'].append(pca_lr_mae)


Set up and Train the Deep Q-Network (DQN)

In [19]:
# 4. Deep Q-Network (DQN) Setup
# Build the DQN model
input_shape = (1,) + X_train.shape[1:]  # Add sequence dimension
nb_actions = 1  # Predicting a single insulin dose value

model = build_dqn_model(input_shape, nb_actions)
dqn = build_dqn_agent(model, nb_actions)

# Reshape X_train and X_test for DQN
X_train_reshaped = np.expand_dims(X_train, axis=1)
X_test_reshaped = np.expand_dims(X_test, axis=1)

# Train the DQN model
dqn.fit(X_train_reshaped, nb_steps=5000, visualize=False, verbose=1)

# Use the model to predict insulin dose values
dqn_pred = model.predict(X_test_reshaped).flatten()

# Evaluate DQN performance
dqn_rmse = np.sqrt(mean_squared_error(y_test, dqn_pred))
dqn_mae = mean_absolute_error(y_test, dqn_pred)
results['Algorithm'].append('Deep Q-Network')
results['RMSE'].append(dqn_rmse)
results['MAE'].append(dqn_mae)

# Convert results into a DataFrame
results_df = pd.DataFrame(results)
print(results_df)


TypeError: Keras symbolic inputs/outputs do not implement `__len__`. You may be trying to pass Keras symbolic inputs/outputs to a TF API that does not register dispatching, preventing Keras from automatically converting the API call to a lambda layer in the Functional Model. This error will also get raised if you try asserting a symbolic input/output directly.

Plot Predictions for All Models

In [ ]:
# Plot predictions for each algorithm
plt.figure(figsize=(10, 6))

# Use test set indices to get the corresponding timestamps
test_timestamps = df.loc[test_idx, 'timestamp']

# Plot actual insulin dose using test set timestamps
plt.plot(test_timestamps, y_test, label='Actual Insulin Dose', color='black', linestyle='--')

# Add predicted values from each model to the plot
plt.plot(test_timestamps, lr_pred, label='Linear Regression', marker='o')
plt.plot(test_timestamps, rf_pred, label='Random Forest', marker='x')
plt.plot(test_timestamps, pca_lr_pred, label='PCA + Linear Regression', marker='s')
plt.plot(test_timestamps, dqn_pred, label='Deep Q-Network', marker='d')

# Customize the plot
plt.xlabel('Timestamp')
plt.ylabel('Insulin Dose')
plt.title('Predicted vs Actual Insulin Dose')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()
